# [네이버 영화 리뷰 워드클라우드](https://blog.naver.com/111ambition/222836972997)

## 필요한 모듈 및 데이터 불러오기

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import urllib.request
# !pip install konlpy → 아래 코드 오류에 대한 선행 코드
from konlpy.tag import Okt
from tqdm import tqdm

In [6]:
# 데이터 불러오기
# urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", filename="pydata/ratings_train.txt")
# urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", filename="pydata/ratings_test.txt")

('ratings_test.txt', <http.client.HTTPMessage at 0x1da94c040d0>)

In [9]:
train_data = pd.read_table('ratings_train.txt') # read_table(): csv, txt 파일을 불러오는 함수
test_data = pd.read_table('ratings_test.txt')
train_data[:5]

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [10]:
test_data[:5]

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0


## Stopwords 및 특수문자 필터링을 위한 준비

In [ ]:
comment_words = ""